<a href="https://colab.research.google.com/github/whatnews72/----/blob/master/5%EC%A3%BC%EC%B0%A8_%EC%8B%A4%EC%8A%B51_seq2seq_encoder_decoder%EB%B2%88%EC%97%AD_Cindy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 정의

*  영문을 입력받아 프랑스어로 번역





# 데이터 세트


*   데이터 세트명 : 타토에바 프로젝트 문장 번역 데이터셋
*   데이터 세트 출처 : http://www.manythings.org/anki/



# 데이터 세트 상세 설명

*   파일은 src, tar, lic 세 개의 열로 구성됩니다.
*   열은 탭으로 구분됩니다.
*   총 227,815개


*   src ; 영어 문장
*   tar : 영어 문장에 대응하는 프랑스어
*   lic : 라이센스(출처)










In [1]:
# 필수 패키지 로드
import os
import shutil
import zipfile
import gc

import pandas as pd
import numpy as np
import tensorflow as tf
import requests


In [2]:
# 학습 데이터 생성
headers ={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}
def download(url, file_name):
    with open(file_name, "wb") as f:
        response = requests.get(url, headers=headers)
        f.write(response.content)


url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
print('zipfilename : ', zipfilename)
download(url, zipfilename)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

zipfilename :  /content/fra-eng.zip


In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))
lines.head(5)

전체 샘플의 개수 : 232736


,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:10000] # 6만개만 실험
lines.sample(10)

,src,tar
4385,We can't go.,Nous ne pouvons pas nous y rendre.
4142,They're bad.,Ils sont mauvais.
4423,We survived!,Nous avons survécu !
3027,Do you fish?,Vous pêchez ?
2291,Is it blue?,Est-ce bleu ?
8411,I'm impulsive.,Je suis impulsif.
7590,He's annoying.,Il est embêtant.
436,Shut up!,Fermez-la !
1361,Keep back.,Restez en retrait.
4425,We survived.,Nous avons survécu.


In [5]:
lines[:10]

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
5,Hi.,Salut.
6,Run!,Cours !
7,Run!,Courez !
8,Run!,Prenez vos jambes à vos cous !
9,Run!,File !


In [6]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
7505,He got caught.,\t Il s'est fait prendre. \n
9581,Tom is stable.,\t Tom est stable. \n
936,Can we go?,\t Pouvons-nous y aller ? \n
3808,It's a song.,\t C'est une chanson. \n
3996,Shall we go?,\t On y va ? \n
4951,Empty it out.,\t Videz-le. \n
674,I saw it.,\t Je l’ai vu. \n
787,Look out!,\t Fais attention. \n
4849,Come on down.,\t Descends. \n
5808,Is it a trap?,\t Est-ce que c'est un piège ? \n


In [7]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [8]:
src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("source 문장의 char 집합 : ", src_vocab_size)
print("target 문장의 char 집합 : ", tar_vocab_size)

source 문장의 char 집합 :  70
target 문장의 char 집합 :  91


In [9]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É']


In [10]:
scr_to_index = dict([(word, i) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i) for i, word in enumerate(tar_vocab)])
print(scr_to_index)
print(tar_to_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '7': 15, '8': 16, '9': 17, ':': 18, '?': 19, 'A': 20, 'B': 21, 'C': 22, 'D': 23, 'E': 24, 'F': 25, 'G': 26, 'H': 27, 'I': 28, 'J': 29, 'K': 30, 'L': 31, 'M': 32, 'N': 33, 'O': 34, 'P': 35, 'Q': 36, 'R': 37, 'S': 38, 'T': 39, 'U': 40, 'V': 41, 'W': 42, 'Y': 43, 'a': 44, 'b': 45, 'c': 46, 'd': 47, 'e': 48, 'f': 49, 'g': 50, 'h': 51, 'i': 52, 'j': 53, 'k': 54, 'l': 55, 'm': 56, 'n': 57, 'o': 58, 'p': 59, 'q': 60, 'r': 61, 's': 62, 't': 63, 'u': 64, 'v': 65, 'w': 66, 'x': 67, 'y': 68, 'z': 69}
{'\t': 0, '\n': 1, ' ': 2, '!': 3, '%': 4, '&': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '5': 14, '8': 15, '9': 16, ':': 17, '?': 18, 'A': 19, 'B': 20, 'C': 21, 'D': 22, 'E': 23, 'F': 24, 'G': 25, 'H': 26, 'I': 27, 'J': 28, 'K': 29, 'L': 30, 'M': 31, 'N': 32, 'O': 33, 'P': 34, 'Q': 35, 'R': 36, 'S': 37, 'T': 38, 'U': 39, 'V': 40, 'W': 41, 'Y': 42,

In [11]:
encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        # 각 char을 정수로 변환
        encoded_line.append(scr_to_index[char])
    encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[26, 58, 9], [26, 58, 9], [26, 58, 9], [26, 58, 9], [27, 52, 9]]


In [12]:
decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :', decoder_input[:5])

target 문장의 정수 인코딩 : [[0, 2, 40, 43, 2, 3, 2, 1], [0, 2, 31, 43, 60, 45, 50, 47, 9, 2, 1], [0, 2, 23, 56, 2, 60, 57, 63, 62, 47, 2, 3, 2, 1], [0, 2, 20, 57, 63, 49, 47, 2, 3, 2, 1], [0, 2, 37, 43, 54, 63, 62, 2, 3, 2, 1]]


In [13]:
decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :', decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[2, 40, 43, 2, 3, 2, 1], [2, 31, 43, 60, 45, 50, 47, 9, 2, 1], [2, 23, 56, 2, 60, 57, 63, 62, 47, 2, 3, 2, 1], [2, 20, 57, 63, 49, 47, 2, 3, 2, 1], [2, 37, 43, 54, 63, 62, 2, 3, 2, 1]]


In [14]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 14
target 문장의 최대 길이 : 61


In [15]:
encoder_input = tf.keras.preprocessing.sequence.pad_sequences(encoder_input, maxlen=max_src_len, padding='post') # 뒤에 0 채우기
decoder_input = tf.keras.preprocessing.sequence.pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = tf.keras.preprocessing.sequence.pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

encoder_input = tf.keras.utils.to_categorical(encoder_input)
decoder_input = tf.keras.utils.to_categorical(decoder_input)
decoder_target = tf.keras.utils.to_categorical(decoder_target)

# 모델링

In [16]:
# Encoder 정의
encoder_inputs = tf.keras.Input(shape=(None, src_vocab_size))
_, state_hidden, state_cell = tf.keras.layers.LSTM(units=16, return_state=True)(encoder_inputs)
encoder_states = [state_hidden, state_cell]

In [17]:
# 모델 생성
model = tf.keras.Model([encoder_inputs], encoder_states)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 70)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ [(None, 16), (None, 16),    │           5,568 │
│                                      │ (None, 16)]                 │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,568 (21.75 KB)

 Trainable params: 5,568 (21.75 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Decoder 정의
decoder_inputs = tf.keras.Input(shape=(None, tar_vocab_size))
decoder_out, _, _ = tf.keras.layers.LSTM(units=16, return_state=True, return_sequences=True)(decoder_inputs, initial_state=encoder_states)
outputs = tf.keras.layers.Dense(tar_vocab_size, activation='softmax')(decoder_out)

In [19]:
# 최종 모델 생성
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 70)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 91)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 16), (None,    │          5,568 │ input_layer[0][0]      │
│                           │ 16), (None, 16)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 16),     │          6,912 │ input_layer_1[0][0],   │
│                           │ (None, 16), (None,     │                │ lstm[0][1], lstm[0][2] │
│                           │ 16)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 91)       │          1,547 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,027 (54.79 KB)

 Trainable params: 14,027 (54.79 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=16, epochs=2, validation_split=0.2)

Epoch 1/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - acc: 0.6221 - loss: 2.4891 - val_acc: 0.7148 - val_loss: 1.0773
Epoch 2/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - acc: 0.7529 - loss: 0.9295 - val_acc: 0.7435 - val_loss: 0.9202


In [21]:
gc.collect()

1038

# 모델 평가

In [22]:
# 학습 데이터 처음 100건을 이용하여 평가
score = model.evaluate(x=[encoder_input[0:100], decoder_input[0:100]], y=decoder_target[0:100])
print('accuracy : ', score[1])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - acc: 0.8560 - loss: 0.5707  
accuracy :  0.8498360514640808


In [23]:
# score 점수를 파일로 저장
fname = 'Week5_1.txt'
f = open('{fname}'  , 'w')
f.write(str(score[1]))
f.close()

# 예측

In [29]:
# 예측용 Encoder 모델 : 학습에서의 encoder_states 이용
encoder_model = tf.keras.Model(encoder_inputs, outputs=encoder_states)

In [30]:
# context 정보 저장 (번역 최초에슨 입력 값이 됨)
decoder_state_input_h = tf.keras.Input(shape=(16,))
decoder_state_input_c = tf.keras.Input(shape=(16,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder 정의 (초기 상태(initial_state)를 이전 시점의 상태를 이용)
decoder_outputs, decoder_state_h, decoder_state_c = tf.keras.layers.LSTM(units=16, return_state=True, return_sequences=True
                                                                         )(decoder_inputs, initial_state=decoder_states_inputs)

# Decoder의 은닉 상태와 셀 상태
decoder_states = [decoder_state_h, decoder_state_c]
decoder_outputs = tf.keras.layers.Dense(tar_vocab_size, activation='softmax')(decoder_outputs)
decoder_model = tf.keras.Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [32]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

NameError: name 'src_to_index' is not defined

In [1]:
index_to_src

NameError: name 'index_to_src' is not defined